In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('Day1').getOrCreate()

In [ ]:
data = spark.read.csv("/mnt/adls/DSG/ingestion/ID/SWAT/dummy/sales_info.csv", header= True)



In [ ]:
data.show()

In [ ]:
data.columns

In [ ]:
data.describe().show()

In [ ]:
type(data)

In [ ]:
type(data['Person'])

In [ ]:
data.select('Person')

In [ ]:
data.select('Person').show()

In [ ]:
data.head(2)

In [ ]:
data.head(2)[0]

In [ ]:
data.select(['Company','Person']).show()

In [ ]:
data.withColumn('newcompany', data['Company']).show()

In [ ]:
data.withColumn('sales_2', data['Sales']*2).show()

In [ ]:
# not an inplace operation

In [ ]:
data.show()

In [ ]:
data2 = data.withColumn('sales_2', data['Sales']*2)

In [ ]:
data2.show()

In [ ]:
# renaming a column

In [ ]:
data.withColumnRenamed('Sales', 'Sales_2020').show()

In [ ]:
display(data)

In [ ]:
display(data.filter("Sales > 500"))

In [ ]:
data.filter("Sales > 500").show()

In [ ]:
data.filter("Sales < 500").select(['Company','Person']).show()

In [ ]:
data.groupBy("Sales").mean().show()

In [ ]:
data.printSchema()

In [ ]:
mod_data = data.withColumn("Sales_new", data["Sales"].cast("double")).drop("Sales")

In [ ]:
mod_data.printSchema()

In [ ]:
mod_data.groupBy("Company").sum("Sales_new").show()

In [ ]:
mod_data.groupBy("Company").mean("Sales_new").show()

In [ ]:
mod_data.groupBy("Company").min("Sales_new").show()

In [ ]:
mod_data.groupBy("Company").max("Sales_new").show()

In [ ]:
mod_data.groupBy("Company").count().show()

In [ ]:
mod_data.orderBy("sales_new").desc().show()

In [ ]:
mod_data.na.drop()

In [ ]:
mod_data.na.fill(0)

In [ ]:
# Linear model

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
data_ecom = spark.read.csv("/mnt/adls/DSG/ingestion/ID/SWAT/dummy/Ecommerce_Customers.csv", header= True)

In [ ]:
display(data_ecom)

In [ ]:
data_ecom.columns

In [ ]:
data_ecom.printSchema()

In [ ]:
from pyspark.sql.types import StructField,StructType,DoubleType,IntegerType, NumericType, StringType

In [ ]:
Schema = StructType([ StructField("Email", StringType()),
                      StructField("Address", StringType()),
                      StructField("Avatar", StringType()),
                      StructField("Avg Session Length", DoubleType()), 
                      StructField("Time on App", DoubleType()),                      
                      StructField("Time on Website", DoubleType()),
                      StructField("Length of Membership", DoubleType()),
                      StructField("Yearly Amount Spent", DoubleType())
                    ])

In [ ]:
data_ecom = spark.read.csv("/mnt/adls/DSG/ingestion/ID/SWAT/dummy/Ecommerce_Customers.csv", header= True, schema=Schema)

In [ ]:
type(data_ecom)

In [ ]:
data_ecom.columns

In [ ]:
display(data_ecom)

In [ ]:
data_ecom.printSchema()

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler( inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'], outputCol = 'features')

In [ ]:
output2 = assembler.transform(data_ecom)

In [ ]:
display(output2)

In [ ]:
output2.head()

In [ ]:
 df_train,df_test = output2.randomSplit([0.9, 0.1],seed = 42)


In [ ]:
output2.count()

In [ ]:
df_test.count()

In [ ]:
 display(df_train)
  

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
model = LinearRegression(featuresCol='features', labelCol='Yearly Amount Spent')

In [ ]:
e_model = model.fit(df_train)

In [ ]:
e_model.coefficients

In [ ]:
e_model.intercept

In [ ]:
modelsummary = e_model.summary

In [ ]:
modelsummary.r2adj

In [ ]:
predy = e_model.transform(df_test)

In [ ]:
display(predy)

In [ ]:
test_res = e_model.evaluate(df_test)

In [ ]:
test_res.rootMeanSquaredError